In [52]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [53]:
history_df = pd.read_json('data/streaming_history.json')
# print(history_df.head())

In [54]:
history_df.to_csv('data/streaming_history.csv', index=False)
print(history_df.shape)

(13509, 21)


In [55]:
history_df.info()
# We can probably drop episode name, episode show name and episode uri

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13509 entries, 0 to 13508
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ts                                 13509 non-null  object 
 1   username                           13509 non-null  object 
 2   platform                           13509 non-null  object 
 3   ms_played                          13509 non-null  int64  
 4   conn_country                       13509 non-null  object 
 5   ip_addr_decrypted                  13509 non-null  object 
 6   user_agent_decrypted               11607 non-null  object 
 7   master_metadata_track_name         13402 non-null  object 
 8   master_metadata_album_artist_name  13402 non-null  object 
 9   master_metadata_album_album_name   13402 non-null  object 
 10  spotify_track_uri                  13402 non-null  object 
 11  episode_name                       20 non-null     obj

In [56]:
spotify_stream_df = history_df[[
    'ts', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name']].copy()
# spotify_stream_df.tail(3)
spotify_stream_df['ts'] = pd.to_datetime(spotify_stream_df['ts']) # convert to datetime

spotify_stream_df.sample(10)

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name
2585,2023-07-05 23:38:49+00:00,6420,Doomsday (with Juice WRLD & Cordae),Lyrical Lemonade
12337,2024-06-01 19:44:36+00:00,150241,Out thë way,Yeat
3662,2023-09-28 01:29:07+00:00,11305,Unforgettable,French Montana
5055,2023-11-09 03:11:54+00:00,0,Breathe,Fearless Soul
7119,2024-01-06 04:18:10+00:00,206840,STARGAZING,Travis Scott
10415,2024-04-16 04:37:42+00:00,265996,Solo,Future
12158,2024-05-29 12:17:19+00:00,166346,inner guidance,Laffey
3894,2023-10-08 04:15:17+00:00,291933,Jimmy Cooks (feat. 21 Savage),Drake
3033,2023-07-28 05:14:54+00:00,35781,20 Min,Lil Uzi Vert
1316,2023-05-29 16:57:47+00:00,104540,Ransom,Lil Tecca


In [57]:
# Unique artists and tracks:
print("Unique Artists: ", len(spotify_stream_df["master_metadata_album_artist_name"].unique()))
print("Unique Tracks: ", len(spotify_stream_df["master_metadata_track_name"].unique()))


Unique Artists:  780
Unique Tracks:  2901


In [58]:
spotify_stream_df['play_time'] = pd.to_datetime(spotify_stream_df['ts'])
spotify_stream_df['year'] = pd.DatetimeIndex(spotify_stream_df['play_time']).year
spotify_stream_df['month'] = pd.DatetimeIndex(spotify_stream_df['play_time']).month
spotify_stream_df['day'] = pd.DatetimeIndex(spotify_stream_df['play_time']).day
spotify_stream_df['weekday'] = pd.DatetimeIndex(spotify_stream_df['play_time']).weekday
spotify_stream_df['hour'] = pd.DatetimeIndex(spotify_stream_df['play_time']).hour
spotify_stream_df['time'] = pd.DatetimeIndex(spotify_stream_df['play_time']).time
spotify_stream_df['day-name'] = spotify_stream_df['play_time'].apply(lambda x: x.day_name())
spotify_stream_df['count'] = 1

In [59]:
spotify_stream_df['time_played'] = pd.to_timedelta(spotify_stream_df['ms_played'], unit='ms')

def hours_played(time):
    return time.seconds / 3600
def minutes_played(time):
    return time.seconds / 60 % 60


spotify_stream_df["time_played_hours"] = spotify_stream_df["time_played"].apply(
    hours_played).round(3)
spotify_stream_df["time_played_minutes"] = spotify_stream_df["time_played"].apply(
    minutes_played).round(3)

In [61]:
spotify_stream_df.tail()

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,play_time,year,month,day,weekday,hour,time,day-name,count,time_played,time_played_hours,time_played_minutes
13504,2024-06-12 22:14:27+00:00,5189,That Fiya,Lil Uzi Vert,2024-06-12 22:14:27+00:00,2024,6,12,2,22,22:14:27,Wednesday,1,0 days 00:00:05.189000,0.001,0.083
13505,2024-06-12 22:14:30+00:00,3157,5!RE,Homixide Gang,2024-06-12 22:14:30+00:00,2024,6,12,2,22,22:14:30,Wednesday,1,0 days 00:00:03.157000,0.001,0.050
13506,2024-06-12 22:16:20+00:00,110926,Go,Ken Carson,2024-06-12 22:16:20+00:00,2024,6,12,2,22,22:16:20,Wednesday,1,0 days 00:01:50.926000,0.031,1.833
13507,2024-06-12 22:19:33+00:00,192213,"Like This (feat. Lil Uzi Vert, Destroy Lonely)",Ken Carson,2024-06-12 22:19:33+00:00,2024,6,12,2,22,22:19:33,Wednesday,1,0 days 00:03:12.213000,0.053,3.200
13508,2024-06-12 23:42:58+00:00,113030,Fighting My Demons,Ken Carson,2024-06-12 23:42:58+00:00,2024,6,12,2,23,23:42:58,Wednesday,1,0 days 00:01:53.030000,0.031,1.883
